In [2]:
from collections import namedtuple

import casadi as ca
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import HTML, display

In [3]:
# Constants
nq = 3 + 3
save_gifs = False
nv = 3
nu = 2 + 3
g = 9.81

# Physical parameters
m = 12.0  # kg (similar to mini cheetah)
L = 0.7  # body length (m)
inertia = 1 / 12 * m * (L**2)  # moment of inertia for rectangular body

leg_length = 0.3  # m
wheel_radius = 0.1  # m

ForwardKinematicsResult = namedtuple(
    "ForwardKinematicsResult",
    [
        "com_x",
        "com_y",
        "knee_x",
        "knee_y",
        "ankle_x",
        "ankle_y",
    ],
)


def forward_kinematics(q):
    lib = ca
    if isinstance(q, np.ndarray):
        lib = np

    if isinstance(q, list) and isinstance(q[0], sp.Symbol):
        lib = sp

    out = dict()
    out["com_x"] = q[0]
    out["com_y"] = q[1]

    phi = q[2]

    theta1 = q[3]
    theta2 = q[4]

    knee_x = out["com_x"] + leg_length * lib.cos(phi - lib.pi / 2 * 1 + theta1)
    knee_y = out["com_y"] + leg_length * lib.sin(phi - lib.pi / 2 * 1 + theta1)

    ankle_x = knee_x + leg_length * lib.cos(phi - lib.pi / 2 * 1 + theta1 + theta2)
    ankle_y = knee_y + leg_length * lib.sin(phi - lib.pi / 2 * 1 + theta1 + theta2)

    out["knee_x"] = knee_x
    out["knee_y"] = knee_y
    out["ankle_x"] = ankle_x
    out["ankle_y"] = ankle_y

    return ForwardKinematicsResult(**out)

In [4]:
class RheaVisualizer:
    def __init__(self, leg_length, wheel_radius):
        self.leg_length = leg_length
        self.wheel_radius = wheel_radius

        # Initialize the figure and axis
        self.fig, self.ax = plt.subplots()
        self.ax.set_aspect("equal")
        self.ax.set_xlim(-1, 2.2)
        self.ax.set_ylim(0.0, 2.5)

        # Initialize plot elements
        (self.com_plot,) = self.ax.plot([], [], "ko", markersize=10, label="COM")
        (self.knee_plots,) = self.ax.plot([], [], "ro", markersize=10, label="Knees")
        (self.ankle_plots,) = self.ax.plot([], [], "go", markersize=10, label="Ankles")
        (self.link_plots,) = self.ax.plot([], [], "b", label="Links")
        (self.force_lines,) = self.ax.plot([], [], "r-", linewidth=2)
        (self.q_trajectory,) = self.ax.plot([], [], "k--", lw=1, alpha=0.5)

        self.ax.plot([-10, 10], [0.77, 0.77], "k--", label="Ground")

        self.wheel_plot = plt.Circle((0, 0), wheel_radius, fill=False, color="b", label="Wheel")
        self.ax.add_patch(self.wheel_plot)

    def set_data(self, q, u=None, q_trajectory=None):
        fk = forward_kinematics(q)

        # Update COM
        self.com_plot.set_data([fk.com_x], [fk.com_y])

        # Update knees
        self.knee_plots.set_data([fk.knee_x], [fk.knee_y])

        # Update ankles
        self.ankle_plots.set_data([fk.ankle_x], [fk.ankle_y])

        # Update links
        self.link_plots.set_data([fk.com_x, fk.knee_x, fk.ankle_x], [fk.com_y, fk.knee_y, fk.ankle_y])

        # Update wheel position
        self.wheel_plot.center = (fk.ankle_x, fk.ankle_y)

        if u is not None:
            F1_x, F1_y = u[:2] / 300
            self.force_lines.set_data(
                [fk.ankle_x, fk.ankle_x + F1_x], [fk.ankle_y - wheel_radius, fk.ankle_y + F1_y - wheel_radius]
            )

        if q_trajectory is not None:
            self.q_trajectory.set_data(q_trajectory[0, :], q_trajectory[1, :])

        plt.draw()

In [ ]:
q = [sp.Symbol("x"), sp.Symbol("y"), sp.Symbol("phi"), sp.Symbol("q3"), sp.Symbol("q4"), sp.Symbol("q5")]
fk = forward_kinematics(q)
jacobian_expr = sp.Matrix([fk.ankle_x, fk.ankle_y]).jacobian(q[:-1])
jacobian = sp.lambdify(q[:-1], jacobian_expr)

print("Leg jacobian:")
display(jacobian_expr)

In [6]:
def euler_integrate(q, v, u, dt, wheel_radius):
    x, y, phi = q[0], q[1], q[2]
    vx, vy, omega = v[0], v[1], v[2]
    theta1, theta2, wheel_angle = q[3], q[4], q[5]
    F1_x, F1_y = u[0], u[1]
    dtheta1, dtheta2, dwheel_angle = u[2], u[3], u[4]

    # Integrate
    x_next = x + vx * dt
    y_next = y + vy * dt
    phi_next = phi + omega * dt

    theta1_next = theta1 + dtheta1 * dt
    theta2_next = theta2 + dtheta2 * dt
    wheel_angle_next = wheel_angle + dwheel_angle * dt

    ax = (F1_x) / m
    ay = (F1_y) / m - g  # gravity

    fk = forward_kinematics(q)

    # Compute position of the feet relative to the COM
    r_x = fk.ankle_x - fk.com_x
    r_y = (fk.ankle_y - wheel_radius) - fk.com_y

    wrench = -F1_x * r_y + F1_y * r_x
    eps = 1 / inertia * wrench

    vx_next = vx + ax * dt
    vy_next = vy + ay * dt
    omega_next = omega + eps * dt

    q_next = np.array([x_next, y_next, phi_next, theta1_next, theta2_next, wheel_angle_next])
    v_next = np.array([vx_next, vy_next, omega_next])

    return q_next, v_next

In [ ]:
# Create state vector [x, y, phi, theta1-4]
q_initial = np.array([1.0, 1.2, 0, -1, 2, 0])
v_initial = np.array([0, 0, 0])

q_final = np.array([0.5, 1.2, 2 * np.pi, -1, 2, 0])
v_final = np.array([0, 0, 0])


visualizer = RheaVisualizer(leg_length, wheel_radius)
visualizer.set_data(q_initial)
plt.show()
q = ca.SX.sym("q", nq)
v = ca.SX.sym("v", nv)
u = ca.SX.sym("u", nu)

dt = 0.02

q_next, v_next = euler_integrate(q, v, u, dt, wheel_radius)

get_next_state = ca.Function("get_next_state", [q, v, u], [q_next, v_next])

opti = ca.Opti()

N = 100

Q = opti.variable(nq, N)
V = opti.variable(nv, N)
U = opti.variable(nu, N - 1)


opti.subject_to(Q[:, 0] == q_initial)
opti.subject_to(V[:, 0] == v_initial)

opti.subject_to(Q[:-1, -1] == q_final[:-1])
opti.subject_to(V[:, -1] == v_final)


def add_friction_cone_constraint(opti, U, k, mu):
    index = [0, 1]
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx >= -mu * fy)
    opti.subject_to(fx <= mu * fy)


def add_zero_force_constraint(opti, U, k):
    index = [0, 1]
    fx = U[index[0], k]
    fy = U[index[1], k]
    opti.subject_to(fx == 0)
    opti.subject_to(fy == 0)


mu = 0.99
for k in range(N - 1):
    q_next, v_next = get_next_state(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

    fk_initial = forward_kinematics(q_initial)
    ry_initial = fk_initial.ankle_y
    fk = forward_kinematics(Q[:, k])
    r_y = fk.ankle_y
    opti.subject_to(Q[1, k] >= ry_initial)
    if k > 20 and k < 55:
        add_zero_force_constraint(opti, U, k)
        opti.subject_to(r_y >= ry_initial + 0.05)
    else:
        add_friction_cone_constraint(opti, U, k, mu)

        J = jacobian(Q[0, k], Q[1, k], Q[2, k], Q[3, k], Q[4, k])
        vel = [None for _ in range(5)]
        vel[0] = V[:, k][0]
        vel[1] = V[:, k][1]
        vel[2] = V[:, k][2]
        vel[3] = U[:, k][2]
        vel[4] = U[:, k][3]
        vel = np.array(vel)

        vel_wheel = U[:, k][4] * wheel_radius

        opti.subject_to(r_y == ry_initial)
        opti.subject_to(vel_wheel == -vel[0])

opti.subject_to(U[0, 0] == 0)
opti.subject_to(U[0, -1] == 0)

cost = 0
for k in range(N - 1):
    q_angles = Q[3:5, k]
    q_angles_default = q_initial[3:5]
    q_angles_diff = q_angles - q_angles_default

    cost += 10000 * (Q[1, k] - q_initial[1]) ** 2

    for i in range(2):
        opti.subject_to(ca.fabs(q_angles_diff[i]) <= np.pi / 4)

    cost += 100 * ca.dot(V[:, k], V[:, k])

    x_coord = Q[0, k]

    cost += 1000 * ca.dot(q_angles_diff, q_angles_diff)
    uext = U[:2, k]
    cost += 111.1 * ca.dot(uext, uext)
    uext = U[2:, k]
    cost += 50000 * ca.dot(uext, uext)
    cost += 20 * ca.fabs(U[4, k])


opti.minimize(cost)


opti.solver("ipopt")
sol = opti.solve()

U_sol = sol.value(U)
print(U_sol[:4, :])
print(U_sol[:4, :] >= -1e-4)

Q_sol = sol.value(Q)


def animate(i):
    visualizer.set_data(Q_sol[:, i], None if i == N - 1 else U_sol[:, i], Q_sol[:2, :i])
    return (visualizer.ax,)


anim = animation.FuncAnimation(visualizer.fig, animate, frames=N, interval=50, blit=False)
plt.close()

html = HTML(anim.to_jshtml())
display(html)